In [436]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras

In [437]:
keras.__version__

'2.2.4-tf'

In [7]:
from tensorflow.keras import layers

In [13]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [17]:
import numpy as np
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))
val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

In [18]:
model.fit(data, labels, epochs=1, batch_size=32, validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
1000/1000 [==============================] - 0s 443us/sample - loss: 20.7471 - accuracy: 0.1040 - val_loss: 23.1116 - val_accuracy: 0.1000


In [6]:
import absl
from absl import flags

In [4]:
flags.DEFINE_string(name='flag_b', default='test', help='an example')

In [7]:
flag_obg = flags.FLAGS
flag_obg.flag_b

UnparsedFlagAccessError: Trying to access flag --flag_b before flags were parsed.

In [389]:
_CSV_COLUMN_NAMES = ['ftime', 'app_ver', 'devid', 'omgid', 'os_type', 'algid', 'flushnum', 'is_new_user', 'network_type',
                    'label', 'vid', 'pubtime', 'tag', 'media', 'cate1', 'cate2', 'duration', 'watch_duration', 'video_quality_score',
                    'age', 'gender', 'province', 'city', 'profession', 'education',
                    "cate1_weight_info", "cate2_weight_info", "media_weight_info", "tag_weight_info",
                    'dev_mode', 'session_cate_feedback', 'quality_score', 'video_source',
                    'vvs', 'expos', 'likes', 'undolikes', 'shares', 'comments', 'dislikes',
                    'session_click', 'small_video_first_recommand', 'small_video_second_recommand']

_CSV_COLUMN_DEFAULTS = [[''], [''], [''], [''], [''], [''], [''], [''], ['0'],
                        [0.0], [''], [''], [''], [''], [''], [''], [0], [0.0], ['0'],
                        [0], ['0'], [0], ['0'], ['0'], ['0'],
                        [''], [''], [''], [''],
                        [''], [''], ['0'], [''],
                        ['0'], ['0'], ['0'], ['0'], ['0'], ['0'], ['0'],
                        [''], [''], ['']]
batch_size = 6

In [401]:
sample_batches = tf.data.experimental.make_csv_dataset(
            "/data1/ceph/lorineluo/fsc/prerank_sample/2020040806/*", batch_size, column_names=_CSV_COLUMN_NAMES, column_defaults=_CSV_COLUMN_DEFAULTS,
            label_name=None, field_delim='\t',
            header=False, num_epochs=1, shuffle=True,
            shuffle_buffer_size=10000, shuffle_seed=None, prefetch_buffer_size=batch_size * 1000,
            num_parallel_reads=6)

In [391]:
for sample in sample_batches.take(2):
    print(sample)

OrderedDict([('ftime', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'2020-04-08 05:01:53', b'2020-04-08 06:53:21',
       b'2020-04-08 05:43:37', b'2020-04-08 05:04:00',
       b'2020-04-08 06:42:06', b'2020-04-08 05:42:54'], dtype=object)>), ('app_ver', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'6.7.0.18223', b'8.0.0.20765', b'8.1.3.25106', b'8.1.3.25106',
       b'8.1.3.20856', b'8.0.0.25081'], dtype=object)>), ('devid', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'864004030456716_460006431672749',
       b'866347042832137_460016566315655',
       b'EF117583-3E17-478C-BDA5-C39E78CC8CE5',
       b'A9A5B52B-F772-4DDA-89B5-F204F5A12EAE',
       b'867474039887988_460021687100654',
       b'354CD4F1-C824-4550-82D6-52EACCAC964C'], dtype=object)>), ('omgid', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'dbe67f67335ee94f3c59be05f844cc80be9a001021281d',
       b'5d580b73288efc45c388f6c6329def650e030010214705',
       b'ec07a13a3bc418454f5bfed545d0376f3e2f00

In [398]:
def parse_csv(sample):
    sparse_list = ['tag', 'cate1_weight_info', 'cate2_weight_info', 'media_weight_info',
                   'tag_weight_info', 'session_cate_feedback', 'session_click', 'algid']
    # label = features['label']
    for i in range(len(sparse_list)):
        sparse = sparse_list[i]
        if sparse == "algid":
            sample[sparse] = string_split_(sample[sparse], "|", ":")
        if sparse in ['cate1_weight_info', 'cate2_weight_info', 'media_weight_info', 'tag_weight_info']:
            sample[sparse] = string_split_(sample[sparse], ",", ":")
        if sparse == 'session_click':
            sample[sparse] = string_split_(sample[sparse], ":")
            sample[sparse] = tf.strings.regex_replace(sample[sparse], ":[0-9.]+", "")
        elif sparse == 'tag':
            sample[sparse] = string_split_(sample[sparse], ",")

    return sample#, label


In [399]:
for i in sample_batches.map(parse_csv).take(6):
    print(i)

OrderedDict([('ftime', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'2020-04-08 06:33:34', b'2020-04-08 05:31:48',
       b'2020-04-08 05:24:35', b'2020-04-08 05:01:08',
       b'2020-04-08 06:31:24', b'2020-04-08 05:57:19'], dtype=object)>), ('app_ver', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'7.9.0.20617', b'8.0.0.25081', b'6.7.0.22106', b'8.1.3.25106',
       b'8.1.3.20856', b'8.1.3.20856'], dtype=object)>), ('devid', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'869480030921948_222101306399751',
       b'86CB6C0C-493D-437D-93FC-3685DFDB616A', b'',
       b'5F94895C-9A49-475E-82B3-09F2F95E3F85', b'862466034771717_',
       b'866571039390839_460075395367800'], dtype=object)>), ('omgid', <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'8963367f38f83843e06a6ef5de5b44f6176a0010213b03',
       b'3bc388994ae1f84470983deaa4eccbbdc2120010114217',
       b'6d93348c3bfae744a9094ad2a9505ac371fd0010110a04',
       b'0477c2d1f1cd5045b6b9adce1f5acef07d39001011430

In [394]:
def string_split_(string, *args):
#     if string == '': return ['']
    tmp = string
    for sep in args:
          tmp = tf.strings.split(tmp, sep=sep)
    if len(args) == 2:
        return tf.squeeze(rg[:, :, 0:1], axis=-1).to_tensor()
    elif len(args) == 1:
        return tmp.to_tensor()
    else:
        logging.error("sample parse error. From string_split!")
        return

In [403]:
import math

In [529]:
tag_buk = tf.ragged.constant([[8066, 20979], 
                              [95326, 29628, 51944, 81762], 
                              [75937], 
                              [21080, 67471, 62535]])
num_buckets = 100000
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal([num_buckets, embedding_size],
                       stddev=1.0 / math.sqrt(embedding_size)))

# Look up the embedding for each word.
# word_buckets = tf.strings.to_hash_bucket_fast(queries, tag_buk)
word_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, tag_buk)  

In [532]:
tag_buk

<tf.RaggedTensor [[8066, 20979], [95326, 29628, 51944, 81762], [75937], [21080, 67471, 62535]]>

In [533]:
tf.reduce_max(word_embeddings, axis=1)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[-0.13534468,  0.4837809 ,  0.0629686 , -0.1767606 ],
       [ 0.27620926,  0.31362125,  0.53890795,  0.522794  ],
       [-0.2553489 ,  0.81139684, -0.18363926,  0.19129485],
       [ 0.51092917,  0.4882665 ,  0.5903944 ,  0.49522674]],
      dtype=float32)>

In [531]:
word_embeddings.to_tensor()

<tf.Tensor: shape=(4, 4, 4), dtype=float32, numpy=
array([[[-0.23738874,  0.04789128,  0.0629686 , -0.1767606 ],
        [-0.13534468,  0.4837809 , -0.17738804, -0.34974152],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.06611683,  0.31362125, -0.1594314 ,  0.522794  ],
        [-0.09160112, -0.2767069 ,  0.11103041, -0.521641  ],
        [-0.02089263,  0.2499787 ,  0.53890795, -0.6084118 ],
        [ 0.27620926, -0.35140675,  0.4630991 , -0.3656848 ]],

       [[-0.2553489 ,  0.81139684, -0.18363926,  0.19129485],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.51092917,  0.12768625,  0.5903944 ,  0.49522674],
        [-0.47368142,  0.4882665 , -0.56546843,  0.40420565],
        [-0.7368358 ,  0.09390552,  0.4107268 ,  0.13245247],
        [ 0. 

In [509]:
queries = tf.ragged.constant([['Who', 'is', 'Dan', 'Smith'],
                              ['Pause', ''],
                              ['Will', 'it', 'rain', 'later', 'today']])

# Create an embedding table.
num_buckets = 1024
embedding_size = 4
embedding_table = tf.Variable(
    tf.random.truncated_normal([num_buckets, embedding_size],
                       stddev=1.0 / math.sqrt(embedding_size)))

# Look up the embedding for each word.
word_buckets = tf.strings.to_hash_bucket_fast(queries, num_buckets)
word_embeddings = tf.ragged.map_flat_values(
    tf.nn.embedding_lookup, embedding_table, word_buckets)  

In [468]:
queries.to_tensor(default_value='2')

<tf.Tensor: shape=(3, 5), dtype=string, numpy=
array([[b'Who', b'is', b'Dan', b'Smith', b'2'],
       [b'Pause', b'', b'2', b'2', b'2'],
       [b'Will', b'it', b'rain', b'later', b'today']], dtype=object)>

In [525]:
word_embeddings.to_tensor()

<tf.Tensor: shape=(3, 5, 6), dtype=float32, numpy=
array([[[ 0.4427225 , -0.04168899,  0.10875478,  0.07619802,
          0.3715867 ,  0.07942362],
        [-0.3523809 ,  0.04780529,  0.13346949, -0.17250948,
         -0.10298278,  0.17040019],
        [-0.2376621 ,  0.44544166,  0.28591365,  0.502637  ,
         -0.32364497,  0.09752981],
        [-0.40692624, -0.24202554,  0.11540721, -0.37696347,
          0.3392739 ,  0.0373866 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]],

       [[-0.4899978 ,  0.5906469 , -0.679474  ,  0.10901653,
          0.2630703 ,  0.3809906 ],
        [ 0.12677379, -0.2343223 , -0.71005493,  0.4026107 ,
          0.2339809 ,  0.411114  ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0. 

In [526]:
word_embeddings

<tf.RaggedTensor [[[0.44272249937057495, -0.04168898984789848, 0.10875478386878967, 0.0761980190873146, 0.37158671021461487, 0.07942362129688263], [-0.3523809015750885, 0.0478052943944931, 0.13346949219703674, -0.17250947654247284, -0.1029827818274498, 0.17040018737316132], [-0.23766210675239563, 0.44544166326522827, 0.2859136462211609, 0.5026370286941528, -0.3236449658870697, 0.0975298061966896], [-0.4069262444972992, -0.24202553927898407, 0.11540721356868744, -0.37696346640586853, 0.3392738997936249, 0.03738659620285034]], [[-0.4899978041648865, 0.5906469225883484, -0.6794739961624146, 0.1090165302157402, 0.263070285320282, 0.380990594625473], [0.12677378952503204, -0.23432229459285736, -0.7100549340248108, 0.4026106894016266, 0.23398089408874512, 0.41111400723457336]], [[-0.4112309515476227, -0.6295347809791565, 0.4219312369823456, -0.02975466288626194, 0.45911669731140137, -0.09644684940576553], [-0.1591784805059433, -0.047092244029045105, -0.1782599687576294, 0.4530091881752014, 0

In [520]:
tf.reduce_max(word_embeddings, axis=1)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[ 0.79919004,  0.9420992 , -0.28558725,  0.7726815 ],
       [ 0.09726047,  0.39728472, -0.3464592 , -0.14726734],
       [ 0.5855968 ,  0.49678063,  0.7498547 ,  0.20903215]],
      dtype=float32)>

In [495]:
word_embeddings.to_tensor().numpy()

array([[[ 0.02767892,  0.4637753 , -0.787799  , -0.06652763],
        [ 0.1046272 ,  0.30237326,  0.23979643,  0.18498978],
        [-0.19888046, -0.1167749 ,  0.5160385 ,  0.7898289 ],
        [-0.34223276,  0.31371424,  0.08879036, -0.3347127 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.3341848 , -0.34178886,  0.9617576 ,  0.15898196],
        [-0.78858435, -0.6059336 , -0.2917831 , -0.10786949],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.00259724, -0.22174446, -0.28356552,  0.3213551 ],
        [ 0.37093925, -0.76262766, -0.11486512, -0.01981083],
        [ 0.7702211 , -0.01087544,  0.24896245, -0.0085072 ],
        [-0.32888013, -0.25380182,  0.5080263 , -0.24032743],
        [ 0.41819963,  0.6648972 , -0.2668504 , -0.06584577]]],
      dtype=float32)

In [478]:
word_buckets.to_tensor(default_value=1)

<tf.Tensor: shape=(3, 5), dtype=int64, numpy=
array([[633, 768, 237, 309,   1],
       [ 28,  79,   1,   1,   1],
       [847, 998, 635,  20, 547]])>

In [482]:
embedding = tf.keras.layers.Embedding(input_dim=1000, output_dim=4, mask_zero=True)
masked_output = embedding(word_buckets.to_tensor(default_value=1))

print(masked_output._keras_mask)

tf.Tensor(
[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [ True  True  True  True  True]], shape=(3, 5), dtype=bool)


In [502]:
tf.reduce_max(masked_output, axis=-1)

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.04331255,  0.04993582,  0.02990608, -0.00509154,  0.0437129 ],
       [ 0.04173045,  0.04582458,  0.0437129 ,  0.0437129 ,  0.0437129 ],
       [-0.01439125,  0.01327525, -0.00953878,  0.04920664,  0.0273915 ]],
      dtype=float32)>

In [487]:
masked_output

<tf.Tensor: shape=(3, 5, 4), dtype=float32, numpy=
array([[[ 0.02211406,  0.01596774,  0.04331255, -0.01857034],
        [ 0.00924046,  0.0342059 ,  0.04993582,  0.04480746],
        [ 0.02990608,  0.02008189,  0.014918  ,  0.00066098],
        [-0.02467504, -0.01429601, -0.00509154, -0.0311796 ],
        [ 0.03521631,  0.0437129 , -0.0386685 ,  0.00437385]],

       [[ 0.01592796, -0.03316347,  0.04173045,  0.04141593],
        [ 0.00306232,  0.00544556,  0.00622668,  0.04582458],
        [ 0.03521631,  0.0437129 , -0.0386685 ,  0.00437385],
        [ 0.03521631,  0.0437129 , -0.0386685 ,  0.00437385],
        [ 0.03521631,  0.0437129 , -0.0386685 ,  0.00437385]],

       [[-0.04888618, -0.02555885, -0.01439125, -0.04323551],
        [ 0.01327525, -0.00949159, -0.01289531, -0.00772697],
        [-0.04587546, -0.03006493, -0.04349918, -0.00953878],
        [ 0.01006569,  0.04808101, -0.03547704,  0.04920664],
        [ 0.01992461, -0.01608391, -0.0391349 ,  0.0273915 ]]],
      dtype=f

In [467]:
word_embeddings.to_tensor()

<tf.Tensor: shape=(3, 5, 4), dtype=float32, numpy=
array([[[ 0.09325559,  0.20185928,  0.48058715, -0.63522637],
        [ 0.08067155, -0.09563014, -0.12459449, -0.15725517],
        [ 0.13839704, -0.28878647,  0.56638366, -0.6747421 ],
        [-0.07063291, -0.10313739,  0.1922413 , -0.01744629],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[-0.17155522, -0.71138036,  0.0332401 ,  0.21782263],
        [-0.30162796, -0.01476775, -0.14967443, -0.20867942],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[-0.37111977, -0.98169535,  0.44905955,  0.8055921 ],
        [ 0.7747296 , -0.21400364, -0.15449202,  0.6795202 ],
        [-0.08976944, -0.62335664,  0.2724769 ,  0.15024318],
        [ 0.5939668 ,  0.2508753 , -0.99932784, -0.47951528],
        [-0.47594008,  0.16289982,  0.94383407, -0.1787903 ]]],
      dtype=f

In [460]:
word_buckets

<tf.RaggedTensor [[633, 768, 237, 309], [28, 79], [847, 998, 635, 20, 547]]>

In [465]:
word_embeddings.to_sparse()

In [413]:
tf.strings.to_hash_bucket_fast(['456'], num_buckets)

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([678])>

In [498]:
word_buckets

<tf.RaggedTensor [[633, 768, 237, 309], [28, 79], [847, 998, 635, 20, 547]]>

In [431]:
tf.strings.to_hash_bucket_strong([['456','e3']], num_buckets + 2, key=[1,3])

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[ 36, 441]])>

In [430]:
tf.strings.to_hash_bucket_strong([['456','e3']], num_buckets + 3, key=[1,3])

<tf.Tensor: shape=(1, 2), dtype=int64, numpy=array([[689, 693]])>

In [420]:
tf.strings.to_number(['2'], out_type='int32')

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>

In [429]:
tf.strings.regex_replace("j0930qq80b5_1585117023_20_45_qqvideo2fsc", "_[0-9.a-z]+", "")

<tf.Tensor: shape=(), dtype=string, numpy=b'j0930qq80b5'>

In [434]:
tf.one_hot([1,2], depth=6)  # output: [3 x 3]

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)>

In [435]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    73728/170498071 [..............................] - ETA: 5:35:11

KeyboardInterrupt: 

In [ ]:
x_train

In [440]:
maxpool = keras.layers.MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')

In [457]:
tf.squeeze(word_embeddings.to_tensor())

<tf.Tensor: shape=(3, 5, 4), dtype=float32, numpy=
array([[[ 0.10481597, -0.12276623, -0.05498414, -0.31741625],
        [ 0.41970077,  0.00544349,  0.15178633, -0.40641513],
        [-0.323907  , -0.440482  , -0.15485954,  0.21026959],
        [ 0.20964214,  0.18055502, -0.05744601, -0.25723785],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[ 0.8933085 ,  0.36102742, -0.17049488,  0.7726348 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[-0.45873052, -0.72609794,  0.19776651, -0.2372479 ],
        [-0.11470318, -0.9879131 ,  0.75520647, -0.33967295],
        [-0.5646406 , -0.2956743 ,  0.53428495, -0.9255426 ],
        [ 0.708027  ,  0.36641765, -0.24257188, -0.07277935],
        [-0.22997417, -0.61905664,  0.26275262,  0.3072654 ]]],
      dtype=f

In [455]:
tf.reshape(word_embeddings.to_tensor(), [3, 4, 5])

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[ 0.10481597, -0.12276623, -0.05498414, -0.31741625,
          0.41970077],
        [ 0.00544349,  0.15178633, -0.40641513, -0.323907  ,
         -0.440482  ],
        [-0.15485954,  0.21026959,  0.20964214,  0.18055502,
         -0.05744601],
        [-0.25723785,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.8933085 ,  0.36102742, -0.17049488,  0.7726348 ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[-0.45873052, -0.72609794,  0.19776651, -0.2372479 ,
         -0.11470318],
        [-0.9879131 ,  0.75520647, -0.33967295, -0.5646406 ,
         -0.2956743 ],
        [ 0.53428495, -0.9255426 ,  0.708027  ,  0.36641765,
         -0.24257188],
        [-0.07277935,

In [444]:
maxpool(word_embeddings.to_tensor())

<tf.Tensor: shape=(3, 2, 4), dtype=float32, numpy=
array([[[ 0.41970077,  0.00544349,  0.15178633, -0.31741625],
        [ 0.20964214,  0.18055502, -0.05744601,  0.21026959]],

       [[ 0.8933085 ,  0.36102742,  0.        ,  0.7726348 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]],

       [[-0.11470318, -0.72609794,  0.75520647, -0.2372479 ],
        [ 0.708027  ,  0.36641765,  0.53428495, -0.07277935]]],
      dtype=float32)>

In [501]:
tf.keras.layers.Input(ragged=True, shape=None, tensor=word_buckets ,name='tag', dtype='int64')

ValueError: You should not pass an EagerTensor to `Input`. For example, instead of creating an InputLayer, you should instantiate your model and directly call it on your input.

In [589]:
word_buckets

<tf.RaggedTensor [[21785, 8320, 62957, 22325], [9212, 97263], [90575, 70438, 90619, 46740, 12451]]>

In [580]:
raw_inputs = [
  [83, 91],
  [73, 8],
  [711, 632, 71]
]

padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(word_buckets,
                                                              padding='post')

print(padded_inputs)

ValueError: `sequences` must be iterable.

In [593]:
word_buckets

<tf.RaggedTensor [[21785, 8320, 62957, 22325], [9212, 97263], [90575, 70438, 90619, 46740, 12451]]>

In [600]:
tf.reduce_max(word_buckets,axis=1)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([62957, 97263, 90619])>

In [603]:
embedding = tf.keras.layers.Embedding(input_dim=1000000, output_dim=4, mask_zero=True)
masked_output = embedding(word_buckets)

print(masked_output._keras_mask.to_tensor())

tf.Tensor(
[[ True  True  True  True False]
 [ True  True False False False]
 [ True  True  True  True  True]], shape=(3, 5), dtype=bool)


In [606]:
tf.reduce_mean(masked_output, axis=1)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[ 0.01162446,  0.00643557, -0.00757216,  0.01408826],
       [-0.01441762,  0.01344983, -0.0151397 ,  0.00735163],
       [ 0.0153331 ,  0.00907104, -0.01344691, -0.00597555]],
      dtype=float32)>

In [607]:
1.89126246e-02-4.77478616e-02

-0.028835236999999996

In [604]:
masked_output.to_tensor()

<tf.Tensor: shape=(3, 5, 4), dtype=float32, numpy=
array([[[ 4.60228585e-02,  1.57513283e-02, -4.39189188e-02,
          3.45584638e-02],
        [-9.39211994e-03, -7.78507441e-04, -4.32975069e-02,
          2.36415304e-02],
        [ 3.34854387e-02, -1.62447616e-03,  1.80086382e-02,
          2.84155123e-02],
        [-2.36183405e-02,  1.23939291e-02,  3.89191620e-02,
         -3.02624702e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 1.89126246e-02, -5.12809679e-03, -2.58194804e-02,
          1.95503235e-05],
        [-4.77478616e-02,  3.20277549e-02, -4.45992872e-03,
          1.46837123e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[ 1.04415305e-02,  2.64839046e-02, -3.12260389e-02,
         -4.511

In [599]:
word_buckets.to_tensor()

<tf.Tensor: shape=(3, 5), dtype=int64, numpy=
array([[21785,  8320, 62957, 22325,     0],
       [ 9212, 97263,     0,     0,     0],
       [90575, 70438, 90619, 46740, 12451]])>

In [594]:
tf.reduce_sum(masked_output, axis=1)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.08201781,  0.08141702, -0.05888664,  0.00222669],
       [ 0.03347281,  0.02262886,  0.03583931,  0.07272591],
       [-0.01432154,  0.0552955 ,  0.03726879,  0.02028055]],
      dtype=float32)>

In [576]:
masked_output

<tf.Tensor: shape=(3, 3, 4), dtype=float32, numpy=
array([[[ 0.02563888, -0.04078964,  0.02294176, -0.03490057],
        [ 0.00176678,  0.01162373,  0.02582831,  0.00280756],
        [ 0.01041443,  0.02797202, -0.01324075, -0.01734489]],

       [[-0.04200233,  0.02029765, -0.04962845,  0.01486461],
        [ 0.03253326, -0.02815368, -0.01510599,  0.0496544 ],
        [ 0.01041443,  0.02797202, -0.01324075, -0.01734489]],

       [[-0.02178112,  0.00966971,  0.03029707, -0.02493211],
        [-0.03916609, -0.01516124,  0.03498826, -0.02480627],
        [ 0.04539676, -0.04303655, -0.04759873, -0.00292101]]],
      dtype=float32)>

In [578]:
0.02563888+0.00176678

0.02740566

In [579]:
-0.04078964+0.01162373

-0.029165910000000003

In [565]:
masks=tf.expand_dims(masked_output._keras_mask, axis=2)
masks

<tf.Tensor: shape=(3, 3, 1), dtype=bool, numpy=
array([[[ True],
        [ True],
        [False]],

       [[ True],
        [ True],
        [False]],

       [[ True],
        [ True],
        [ True]]])>

In [567]:
tf.keras.layers.Masking(mask_value=0.,
                                  input_shape=(3, 4))

In [577]:
outputs = tf.keras.layers.GlobalAvgPool1D()(masked_output)
outputs

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[ 0.01370283, -0.01458296,  0.02438503, -0.01604651],
       [-0.00473453, -0.00392802, -0.03236721,  0.03225951],
       [-0.00518349, -0.01617602,  0.00589553, -0.01755313]],
      dtype=float32)>

In [637]:
tag_shared_embedding = tf.keras.layers.Embedding(100000, 4, name='tag_shared_embedding', mask_zero=True)

In [653]:
word_buckets

<tf.RaggedTensor [[21785, 8320, 62957, 22325], [9212, 97263], [90575, 70438, 90619, 46740, 12451]]>

In [652]:
word_buckets.shape

TensorShape([3, None])

In [642]:
td=tag_shared_embedding(word_buckets)
td#.to_tensor()

<tf.RaggedTensor [[[-0.03667619079351425, -0.018729127943515778, -0.00038436800241470337, -0.03889193385839462], [-0.04279431328177452, -0.04780362918972969, 0.0072879791259765625, 0.009880304336547852], [-0.005096804350614548, -0.007333111017942429, -0.02851036749780178, -0.01179875060915947], [0.007532872259616852, 0.008748233318328857, 0.0181715227663517, -0.03662774711847305]], [[0.012711334973573685, -0.028877699747681618, 0.0053331367671489716, -0.03790942579507828], [-0.01294407993555069, 0.004368424415588379, -0.024295305833220482, 0.025109197944402695]], [[0.028396356850862503, -0.022460950538516045, 0.03332412615418434, -0.017166338860988617], [-0.00682387501001358, -0.026720428839325905, -0.01673060655593872, -0.02043016068637371], [0.044156838208436966, -0.037962090224027634, 0.02057698741555214, 0.005271852016448975], [0.049044255167245865, 0.03057250753045082, 0.037478458136320114, 0.025405947118997574], [-0.017199836671352386, 0.009608875960111618, 0.0027819983661174774,

In [651]:
tss

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[-0.01925861, -0.01627941, -0.00085881, -0.01935953],
       [-0.00011637, -0.01225464, -0.00948108, -0.00640011],
       [ 0.01951475, -0.00939242,  0.01548619, -0.00383584]],
      dtype=float32)>

In [650]:
tss=tf.reduce_mean(td, axis=1)
tf.reshape(tss, [-1])

<tf.Tensor: shape=(12,), dtype=float32, numpy=
array([-0.01925861, -0.01627941, -0.00085881, -0.01935953, -0.00011637,
       -0.01225464, -0.00948108, -0.00640011,  0.01951475, -0.00939242,
        0.01548619, -0.00383584], dtype=float32)>

In [614]:
tag_shared_embedding(word_buckets.to_tensor()[0])

<tf.Tensor: shape=(5, 8), dtype=float32, numpy=
array([[ 0.03335372, -0.02145877,  0.03339365,  0.01726082,  0.01184953,
         0.03866493, -0.02065616, -0.02423563],
       [-0.02449707,  0.01659865, -0.02458121,  0.03044927, -0.01967942,
        -0.00633129,  0.04188016,  0.0358826 ],
       [-0.04990908, -0.04027251, -0.04669862,  0.00825089, -0.02803978,
         0.0236613 ,  0.00990739, -0.03768194],
       [-0.017327  ,  0.01863809, -0.02946594, -0.02743986,  0.02898511,
         0.00531839,  0.01411397,  0.02145595],
       [ 0.01231895, -0.03689716, -0.04438584, -0.00258059,  0.02357442,
        -0.00912272,  0.02173406, -0.03583552]], dtype=float32)>

In [636]:
tf.reduce_mean(wd, axis=1)

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([23077, 21295, 62164])>

In [635]:
wd=word_buckets.to_tensor()
wd

<tf.Tensor: shape=(3, 5), dtype=int64, numpy=
array([[21785,  8320, 62957, 22325,     0],
       [ 9212, 97263,     0,     0,     0],
       [90575, 70438, 90619, 46740, 12451]])>

In [633]:
item_dense = tf.keras.layers.concatenate([wd, wd], name='item_dense', axis=-1)
item_dense

<tf.Tensor: shape=(3, 10), dtype=int64, numpy=
array([[21785,  8320, 62957, 22325,     0, 21785,  8320, 62957, 22325,
            0],
       [ 9212, 97263,     0,     0,     0,  9212, 97263,     0,     0,
            0],
       [90575, 70438, 90619, 46740, 12451, 90575, 70438, 90619, 46740,
        12451]])>

In [50]:
import numpy as np
from scipy.special import softmax
from sklearn.preprocessing import normalize

In [70]:
data = np.random.random((10, 4))

In [71]:
data

array([[0.58451664, 0.87499024, 0.98573569, 0.5546548 ],
       [0.79684509, 0.89276585, 0.2758421 , 0.23401034],
       [0.65873929, 0.02465913, 0.3910368 , 0.78605045],
       [0.05085884, 0.77107626, 0.86438269, 0.2944315 ],
       [0.86994728, 0.24741204, 0.87041756, 0.03485488],
       [0.65983937, 0.87243269, 0.50199847, 0.64481019],
       [0.37582619, 0.54784951, 0.46469105, 0.33414523],
       [0.08072885, 0.6225409 , 0.64168146, 0.40045993],
       [0.98519533, 0.99369122, 0.13875781, 0.5362286 ],
       [0.45158042, 0.67885292, 0.1551835 , 0.11600967]])

In [76]:
logits = softmax(np.dot(data, data.T), axis=1)
print("max index for 0-10:", np.argmax(logits, axis=1))

max index for 0-10: [0 8 0 0 0 0 0 0 8 8]


In [73]:
data_norm = normalize(data, axis=1, norm='l2')
print(np.sum(data_norm * data_norm, axis=1))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [77]:
logits = softmax(np.dot(data_norm, data_norm.T), axis=1)
print("max index for 0-10:", np.argmax(logits, axis=1))

max index for 0-10: [0 1 2 3 4 5 6 7 8 9]


In [41]:
norm = np.sqrt(np.sum(data*data, axis=1))
norm = norm.reshape([-1, 1])
norm

array([[1.01866043],
       [1.19776899],
       [1.31255241],
       [1.23743219],
       [1.22614479],
       [1.18916124],
       [1.15434386],
       [1.67497524],
       [0.59221959],
       [1.47055146]])

In [42]:
data1 = data / norm
data

array([[0.43797732, 0.17330583, 0.00207111, 0.90321965],
       [0.41967696, 0.53763247, 0.44421104, 0.87872049],
       [0.31606034, 0.75399709, 0.72216598, 0.72997559],
       [0.41345205, 0.74535425, 0.77695357, 0.44842617],
       [0.29043376, 0.66029663, 0.3969882 , 0.9085637 ],
       [0.92047193, 0.52171879, 0.52058082, 0.15375624],
       [0.45548249, 0.9415205 , 0.44722163, 0.19641131],
       [0.99120226, 0.87349564, 0.29768518, 0.98562113],
       [0.40858535, 0.0672959 , 0.35294188, 0.233849  ],
       [0.6193961 , 0.94334069, 0.9064489 , 0.25947793]])